In [1]:
from zs6d import ZS6D
import os
import json

with open(os.path.join("./dino_pose_configs/cfg_lmo_inference_bop.json"), "r") as f:
    config = json.load(f)
    

pose_estimator = ZS6D(config['templates_gt_path'], config['norm_factor_path'])

/home/philipp/anaconda3/envs/zs6d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/philipp/.cache/torch/hub/facebookresearch_dino_main
100%|██████████| 13/13 [00:06<00:00,  1.93it/s]

Preparing templates and loading of extractor is done!


In [2]:
with open(os.path.join(config['gt_path']), 'r') as f:
    data_gt = json.load(f)

In [3]:
import cv2
from PIL import Image
import pose_utils.img_utils as img_utils
import numpy as np
import time

img_id = '3'


for i in range(5):
    obj_number = i

    obj_name = data_gt[img_id][obj_number]['obj_name']
    obj_id = data_gt[img_id][obj_number]['obj_id']
    cam_K = np.array(data_gt[img_id][obj_number]['cam_K']).reshape((3,3))
    bbox = data_gt[img_id][obj_number]['bbox_visib']

    img_path = os.path.join(config['dataset_path'], data_gt[img_id][obj_number]['img_name'].split("./")[-1])
    img = Image.open(img_path)

    mask = data_gt[img_id][obj_number]['mask_sam']
    mask = img_utils.rle_to_mask(mask)
    mask = mask.astype(np.uint8)

    start_time = time.time()

    R_est, t_est = pose_estimator.get_pose(img, obj_name, obj_id, mask, cam_K, bbox=None)

    end_time = time.time()

    print(f"Pose estimation time: {end_time-start_time}")
    print(f"R_est: {R_est}")
    print(f"t_est: {t_est}")




Preparing: 0.8863692283630371
matching: 0.09585213661193848
running k-means on cuda..


[running kmeans]: 1it [00:00, 507.17it/s, center_shift=0.000000, iteration=1, tol=0.000100]

Size: 74
local correpondences: 1.0089409351348877
Error while local correspondence matching
Pose estimation time: 1.9917280673980713
R_est: None
t_est: None
Preparing: 0.04976773262023926
matching: 0.08985424041748047


running k-means on cuda..


[running kmeans]: 15it [00:00, 44.77it/s, center_shift=0.000000, iteration=15, tol=0.000100]


...

Preparing: 0.04946732521057129
matching: 0.09186959266662598
running k-means on cuda..


[running kmeans]: 11it [00:00, 48.73it/s, center_shift=0.000000, iteration=11, tol=0.000100]


Size: 91
local correpondences: 1.2754464149475098
Error while local correspondence matching
Pose estimation time: 1.4174065589904785
R_est: None
t_est: None
Preparing: 0.04972648620605469
matching: 0.08923006057739258
running k-means on cuda..


[running kmeans]: 5it [00:00, 58.07it/s, center_shift=0.000000, iteration=5, tol=0.000100]

Size: 58
local correpondences: 1.1829650402069092
Error while local correspondence matching
Pose estimation time: 1.3224623203277588
R_est: None
t_est: None


In [1]:
import trimesh
from tools.ply_file_to_3d_coord_model import convert_unique


mesh_path = "/mnt/Littleboy/ZS6D/templates/linemod_desc/models/obj_000001.ply"

out_path = "obj_000001_xyz.ply"

convert_unique(mesh_path, out_path)


(39.819958, 40.709934, 56.63465, -1.8856587, -1.9103358, -10.750154)

In [18]:
from plyfile import PlyData, PlyElement

model_path = '/mnt/Littleboy/ZS6D/templates/ycbv_desc/models_xyz/obj_000010.ply'

model_path_working = '/mnt/Littleboy/ZS6D/templates/ycbv_desc/models_xyz_working/obj_000010.ply'

model_path_before = '/mnt/Littleboy/ZS6D/templates/ycbv/models/obj_000010.ply'

plydata_before = PlyData.read(model_path_before)

plydata_not_working = PlyData.read(model_path)

plydata_working = PlyData.read(model_path_working)

In [19]:
plydata_before.elements[0].data

array([(-47.551, -60.295,   5.166, -5.22226  , -1.919974 ,  2.89706 , 0.896 , 0.1008),
       (-49.817,  -5.815,  -8.504, -5.452106 ,  0.9285001, -2.978487, 0.1156, 0.6888),
       (-30.09 , -85.954, -12.587, -0.1040842, -3.365586 , -4.786951, 0.3977, 0.8282),
       ...,
       (  9.81 ,  80.452,  11.273,  0.4151828,  0.9106925,  2.366138, 0.5008, 0.6881),
       (  7.996,  84.863,   9.165, -0.1622982,  2.499315 ,  2.925677, 0.4943, 0.7013),
       ( 10.651,  83.125,  10.057,  0.4254326,  0.7842768,  1.947044, 0.6062, 0.5532)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('nx', '<f4'), ('ny', '<f4'), ('nz', '<f4'), ('texture_u', '<f4'), ('texture_v', '<f4')])

In [20]:
plydata_working.elements[0].data

memmap([(-47.551, -60.295,   5.166,  71,  36, 167, 255),
        (-49.817,  -5.815,  -8.504,  67, 104,  77, 255),
        (-30.09 , -85.954, -12.587, 103,   4,  50, 255), ...,
        (  9.81 ,  80.452,  11.273, 174, 212, 208, 255),
        (  7.996,  84.863,   9.165, 171, 218, 194, 255),
        ( 10.651,  83.125,  10.057, 176, 216, 200, 255)],
       dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'), ('alpha', 'u1')])

In [25]:
plydata_not_working.elements[0].data

array([(-47.551, -60.295,   5.166, -5.22226  , -1.919974 ,  2.89706 , 0.896 , 0.1008),
       (-49.817,  -5.815,  -8.504, -5.452106 ,  0.9285001, -2.978487, 0.1156, 0.6888),
       (-30.09 , -85.954, -12.587, -0.1040842, -3.365586 , -4.786951, 0.3977, 0.8282),
       ...,
       (  9.81 ,  80.452,  11.273,  0.4151828,  0.9106925,  2.366138, 0.5008, 0.6881),
       (  7.996,  84.863,   9.165, -0.1622982,  2.499315 ,  2.925677, 0.4943, 0.7013),
       ( 10.651,  83.125,  10.057,  0.4254326,  0.7842768,  1.947044, 0.6062, 0.5532)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('nx', '<f4'), ('ny', '<f4'), ('nz', '<f4'), ('texture_u', '<f4'), ('texture_v', '<f4')])

In [2]:
mesh = trimesh.load(out_path)

mesh.show()